# Azure AI Foundry - Local Tool / Function Calling

This notebook demonstrates how to create an agent which is using local function calling (or tool calling) to provide completion.
![](../media/img/FunctionCalling.png)

Two local functions:

- GetSportEventWinner()
- GetSportEventResults()

are defined and provided to the agent as tools where execution can be requested. 

## Step 1 - Load Environment

Two communicate with the Azure AI Foundry Agent Service two parameters are loaded from a configuration file. 

- foundryEndpoint: Endpoint of a created AI Foundry Project. 
- foundryModel: Deployed LLM model

The Foundry Project can be created using by deploying the [BICEP file 'setup.bicep'](../setup/setup.bicep) using the [../setup/setup.azcli - Azure CLI script](../setup/setup.azcli). Necessary parameter can be provided in the [../setup/parameters.json](../setup/parameters.json).




In [ ]:
#r "nuget: DotNetEnv, 3.1.1"
#r "nuget: Azure.AI.Agents.Persistent, 1.0.0"
#r "nuget: Azure.Identity, 1.14.1"

using DotNetEnv;

string configurationFile = @"../config/config.env";
Env.Load(configurationFile);

string foundryEndpoint = Environment.GetEnvironmentVariable("SDK_FOUNDRY_ENDPOINT") ?? "Foundry endpoint not found";
string foundryModelDeployment = Environment.GetEnvironmentVariable("SDK_FOUNDRY_DEPLOYMENTNAME") ?? "Foundry model deployment not found";

Console.WriteLine("Configuration loaded: ");


Installed Packages Azure.AI.Agents.Persistent, 1.0.0 Azure.Identity, 1.14.1 DotNetEnv, 3.1.1

Configuration loaded: 


## Step 2 - Create PersistentAgentsClient

To authenticate against the Azure AI Foundry Agents Service 
- Azure CLI credentials or
- Visual Studio credentials
can be used.

Ensure that you're logged into Azure using e.g. the Azure CLI by executing `az login`

In [4]:
using Azure.Identity;
using Azure.AI.Agents.Persistent;

// Ensure that az login has been executed in the terminal
DefaultAzureCredentialOptions defaultAzureCredentialOptions = new DefaultAzureCredentialOptions
{
    ExcludeInteractiveBrowserCredential = true,
    ExcludeVisualStudioCredential = false,
    ExcludeEnvironmentCredential = true,
    ExcludeManagedIdentityCredential = true,
    ExcludeAzureCliCredential = false
};
DefaultAzureCredential defaultAzureCredential = new DefaultAzureCredential(defaultAzureCredentialOptions);

PersistentAgentsClient persistedAgentsClient = new PersistentAgentsClient(foundryEndpoint, defaultAzureCredential);


Console.WriteLine($"PersistentAgentsClient crated ...");

PersistentAgentsClient crated ...


## Step 3 - Define local functions

The functions `GetSportEventWinner` and `GetSportEventResults` in combination with responding method and parameter signatures are defined.

Both functions simulate local functionality to retrieve results from e.g. protected databases.

In [5]:
using System.Text.Json;

async Task<string> GetSportEventWinner(string sportEvent, string sportEventYear) => await Task.FromResult("Flying Munich Dolphins");
async Task<string> GetSportEventResults(string sportEvent, string sportEventYear) => await Task.FromResult($"Results for {sportEvent} in {sportEventYear}: 35:10");

FunctionToolDefinition getSportEventWinnerTool = new(
    name: "getSportEventWinner",
    description: "Gets the winner of a sport event.",
    parameters: BinaryData.FromObjectAsJson(
        new
        {
            Type = "object",
            Properties = new
            {
                SportEvent = new
                {
                    Type = "string",
                    Description = "The name of the sport event, e.g. 'Super Sport Event'.",
                },
                SportEventYear = new
                {
                    Type = "string",
                    Description = "The year of the sport event, e.g. '2025'.",
                },
            },
            Required = new[] { "sportEvent", "sportEventYear" },
        },
        new JsonSerializerOptions() { PropertyNamingPolicy = JsonNamingPolicy.CamelCase })
);

FunctionToolDefinition getSportEventResultsTool = new(
    name: "getSportEventResults",
    description: "Gets the results of a sport event.",
    parameters: BinaryData.FromObjectAsJson(
        new
        {
            Type = "object",
            Properties = new
            {
                SportEvent = new
                {
                    Type = "string",
                    Description = "The name of the sport event, e.g. 'Super Sport Event'.",
                },
                SportEventYear = new
                {
                    Type = "string",
                    Description = "The year of the sport event, e.g. '2025'.",
                },
            },
            Required = new[] { "sportEvent", "sportEventYear" },
        },
        new JsonSerializerOptions() { PropertyNamingPolicy = JsonNamingPolicy.CamelCase })
);

Console.WriteLine("Local functions/tools with definition created ...");


Local functions/tools with definition created ...


## Step 4 - Helper Function

The helper function `CallInternalFunction` supports calling local functions whenever the LLM requests executing them (pls. use Reflection for production grade solutions).

In [6]:
async Task<ToolOutput?> CallInternalFunction(RequiredToolCall toolCall)
{
    
    string toolCallResult = ""; 
    if (toolCall is RequiredFunctionToolCall functionToolCall)
    {
        using JsonDocument argumentsJson = JsonDocument.Parse(functionToolCall.Arguments);
        Console.WriteLine($"Function {functionToolCall.Name}");

        switch (functionToolCall.Name)
        {
            case "getSportEventWinner":
                string sportEvent = argumentsJson.RootElement.GetProperty("sportEvent").GetString() ?? string.Empty;
                string sportEventYear = argumentsJson.RootElement.GetProperty("sportEventYear").GetString() ?? string.Empty;
                toolCallResult = await GetSportEventWinner(sportEvent, sportEventYear);
                break;
            case "getSportEventResults":
                sportEvent = argumentsJson.RootElement.GetProperty("sportEvent").GetString() ?? string.Empty;
                sportEventYear = argumentsJson.RootElement.GetProperty("sportEventYear").GetString() ?? string.Empty;
                toolCallResult = await GetSportEventResults(sportEvent, sportEventYear);
                break;
            default:
                Console.WriteLine($"Function {functionToolCall.Name} is not implemented.");
                toolCallResult = "";
                break;
        }
    }
    return new ToolOutput(toolCall.Id, toolCallResult);
}

Console.WriteLine("Internal function call handler created ...");

Internal function call handler created ...


## Step 5 - Define Agent

An agent is defined with instruction to use proviced tools to fulfill it's tasks. The definition for: 

- getSportEventWinnerTool and
- getSportEventResultsTool

are provided to the agent

In [7]:
string systemMessage = @"
    You provide information about Sport Events.
    You focus on the question and don't provie any additional information.
    Use the provided functions to help answer questions.
";

PersistentAgent agent = await persistedAgentsClient.Administration.CreateAgentAsync(
    model: foundryModelDeployment,
    name: "GetSportNewsKiosk",
    instructions: systemMessage,
    tools: [
        getSportEventWinnerTool,
        getSportEventResultsTool,
    ]
);
Console.WriteLine($"Agent {agent.Name} created with model {foundryModelDeployment} ...");

Agent GetSportNewsKiosk created with model gpt-4.1-mini ...


## Step 6 - Create and Run Thread

A `PersistentAgentThread` is created with the ***Who won the Super Sports Championship 2025? What were the results?*** initial user message.

A `ThreadRun' object is created connectiong the created Thread with the previously defined agent.

In [8]:
PersistentAgentThread persistentAgentThread = persistedAgentsClient.Threads.CreateThread();

persistedAgentsClient.Messages.CreateMessage(
    persistentAgentThread.Id,
    MessageRole.User,
    "Who won the Super Sport Event in 2025? What were the results?"
);

ThreadRun threadRun = persistedAgentsClient.Runs.CreateRun(persistentAgentThread.Id, agent.Id);

Console.WriteLine($"Thread run {threadRun.Id} created for agent {agent.Name} ...");


Thread run run_kKtjj9cpAlCNJGTsi0UpHnhT created for agent GetSportNewsKiosk ...


## Step 7 - Local Function Calling

On request of the LLM local functions are called and results are provided to the LLM.

In [11]:
using Azure;

do
{
    await Task.Delay(500);

    threadRun = persistedAgentsClient.Runs.GetRun(persistentAgentThread.Id, threadRun.Id);

    if (
        threadRun.Status == RunStatus.RequiresAction
        &&
        threadRun.RequiredAction is SubmitToolOutputsAction submitToolOutputsAction
    )
    {
        List<ToolOutput> toolOutputs = new List<ToolOutput>();
        foreach (RequiredToolCall toolCall in submitToolOutputsAction.ToolCalls)
        {
            ToolOutput? toolOutput = await CallInternalFunction(toolCall);
            if (toolOutput != null)
            {
                toolOutputs.Add(toolOutput);
            }
        }
        threadRun = persistedAgentsClient.Runs.SubmitToolOutputsToRun(threadRun, toolOutputs);
    }
}

while (threadRun.Status == RunStatus.Queued
    || threadRun.Status == RunStatus.InProgress
    || threadRun.Status == RunStatus.RequiresAction);

Console.WriteLine($"Thread run '{threadRun.Id}' completed with status '{threadRun.Status}'...");


Thread run 'run_kKtjj9cpAlCNJGTsi0UpHnhT' completed with status 'completed'...


## Step 8 - Retrieve Thread Messages

After finishing `ThreadRun` all thread messages are retrieved and shown.

In [12]:
Pageable<PersistentThreadMessage> persistentThreadMessages = persistedAgentsClient.Messages.GetMessages(
    threadId: persistentAgentThread.Id,
    order: ListSortOrder.Ascending
);

Console.WriteLine($"Thread run {threadRun.Id} completed ...");
foreach (PersistentThreadMessage persistentThreadMessage in persistentThreadMessages)
{
    foreach (MessageTextContent messageTextContent in persistentThreadMessage.ContentItems.OfType<MessageTextContent>())
    {
        Console.WriteLine($"\t{persistentThreadMessage.Role}: {messageTextContent.Text}");
    }
}

Thread run run_kKtjj9cpAlCNJGTsi0UpHnhT completed ...
	user: Who won the Super Sport Event in 2025? What were the results?
	assistant: The winner of the Super Sport Event in 2025 was the Flying Munich Dolphins. The results for the event were 35:10.


## Step 8 - Housekeeping

Thread and agent are removed.

In [13]:
persistedAgentsClient.Threads.DeleteThread(threadId: persistentAgentThread.Id);
persistedAgentsClient.Administration.DeleteAgent(agentId: agent.Id);

Console.WriteLine("Thread and agent deleted ...");


Thread and agent deleted ...
